<a href="https://colab.research.google.com/github/yairrinat/Survey_Analysis/blob/main/survey_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this short script I plan to analyze the survey data we got from the WMO

# **This is a new version**


In [22]:
# load and install pakages
%pip install google_trans_new
from google_trans_new import google_translator  

import pandas as pd


In [25]:
#this is only a try
translator = google_translator()  
translator.translate("שלום לכולם",lang_tgt="en")


google_new_transError: ignored

# Loading excels:

In [21]:
# link to excel file:
url = "https://github.com/yairrinat/Survey_Analysis/blob/main/WMO_SURVEY.xlsx?raw=true"

#load personal data:
df_personal = pd.read_excel(url,index_col = 0,header = 0, sheet_name = "Person details")
df_personal = df_personal.iloc[1:,:] # remove first row
#print(df_personal)

#load staff data:
df_staff = pd.read_excel(url,index_col=0,header = 0, sheet_name = "STAFF")
list_ind = df_staff.index.to_list()
list_ind[0] = "secondary_title"
df_staff.index= list_ind
#df_staff.head(10)

#load ... to be continued later

In [20]:
# detact and change language 
translaor = google_translator()
df_new = df_staff.applymap(lambda x: translator.translate(x,lang_tgt="en"))

df_staff.head(10)
df_new.head(10)

google_new_transError: ignored

,Please provide the total number of staff in your institution/organization by gender:,Unnamed: 2,"Please provide the number of staff working in the institution/organization, in the following areas along with their gender distribution: (Female/Male)",Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,"Please provide the number of staff in your institution/organization with a university degree, along with their gender distribution:",Unnamed: 9,Please provide the number of staff in your institution/organization by the following age categories:,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Please indicate the trend of the staffing figures during the recent 3-5 years:,Unnamed: 16
secondary_title,Female:,Male:,Management:,Hydrological Modeling and Forecasting:,Information and Communication Technology:,Technician:,"Support (administration, security, general sup...",Female:,Male:,Less than 20 years:,21-30 years:,31-40 years:,41-50 years:,Over 51 years:,Response,Other (please specify)
Albania,15,25,6,8,2,6,4,15,20,0,5,5,20,15,Steadily decreasing,NaN
Argentina,95,146,40 / 26,38 / 61,0 / 0,13 / 40,4 / 19,50,73,0,5,45,45,146,No significant year-to-year change,NaN
Australia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Australia,525,1083,NaN,47 flood forecasting,NaN,NaN,NaN,NaN,NaN,0,1,14,16,16,No significant year-to-year change,NaN
Bahamas,147,238,"65, Female = 36, Male = 29",2,NaN,NaN,NaN,87,79,2,44,96,109,134,Other (please specify),"Staffing compliment is decreasing, based on em..."
Bangladesh,1,13,700/2800,1/13,50/450,1000/4000,200/800,1000,4000,0%,15%,30%,35%,20%,Don't know,NaN
Belarus,783,358,NaN,4/1,NaN,NaN,NaN,206,46,32,74,73,67,134,Без существенных изменений от года к году,NaN
Belgium,NaN,NaN,NaN,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,En augmentation constante,NaN
Belize,3,3,1,1,1,4,1,1,NaN,NaN,2,3,1,NaN,No significant year-to-year change,NaN


In [16]:
df_new.head(10)

,Please provide the total number of staff in your institution/organization by gender:,Unnamed: 2,"Please provide the number of staff working in the institution/organization, in the following areas along with their gender distribution: (Female/Male)",Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,"Please provide the number of staff in your institution/organization with a university degree, along with their gender distribution:",Unnamed: 9,Please provide the number of staff in your institution/organization by the following age categories:,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Please indicate the trend of the staffing figures during the recent 3-5 years:,Unnamed: 16
secondary_title,Female:,Male:,Management:,Hydrological Modeling and Forecasting:,Information and Communication Technology:,Technician:,"Support (administration, security, general sup...",Female:,Male:,Less than 20 years:,21-30 years:,31-40 years:,41-50 years:,Over 51 years:,Response,Other (please specify)
Albania,15,25,6,8,2,6,4,15,20,0,5,5,20,15,Steadily decreasing,male
Argentina,95,146,40/26,38/61,0/0,13/40,4/19,50,73,0,5,45,45,146,No significant year-to-year change,male
Australia,male,male,male,male,male,male,male,male,male,male,male,male,male,male,male,male
Australia,525,1083,male,47 flood forecasting,male,male,male,male,male,0,1,14,16,16,No significant year-to-year change,male
Bahamas,147,238,"65, Female = 36, Male = 29",2,male,male,male,87,79,2,44,96,109,134,Other (please specify),"Staffing compliment is decreasing, based on em..."
Bangladesh,1,13,700/2800,1/13,50/450,1000/4000,200/800,1000,4000,0%,15%,30%,35%,20%,Don't know,male
Belarus,783,358,male,4/1,male,male,male,206,46,32,74,73,67,134,Without significant changes from year to year,male
Belgium,male,male,male,1,5,male,male,male,male,male,male,male,male,male,Constantly,male
Belize,3,3,1,1,1,4,1,1,male,male,2,3,1,male,No significant year-to-year change,male
